In [1]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import pickle

In [2]:
models_names = ['cloudyu/Mixtral_34Bx2_MoE_60B', 'jondurbin/nontoxic-bagel-34b-v0.2', 'jondurbin/bagel-dpo-34b-v0.2', 'moreh/MoMo-70B-LoRA-V1.4', 'TomGrc/FusionNet_linear', 'DopeorNope/SOLARC-M-10.7B', 'cloudyu/Mixtral_11Bx2_MoE_19B', 'TomGrc/FusionNet', 'DopeorNope/SOLARC-MOE-10.7Bx6', 'DopeorNope/SOLARC-MOE-10.7Bx4', 'bhavinjawade/SOLAR-10B-OrcaDPO-Jawade', 'AA051610/testtest', 'VAGOsolutions/SauerkrautLM-SOLAR-Instruct', 'upstage/SOLAR-10.7B-Instruct-v1.0', 'fblgit/UNA-SOLAR-10.7B-Instruct-v1.0', 'kyujinpy/Sakura-SOLRCA-Math-Instruct-DPO-v2', 'kyujinpy/Sakura-SOLAR-Instruct-DPO-v2', 'kyujinpy/Sakura-SOLRCA-Math-Instruct-DPO-v1', 'fblgit/UNA-POLAR-10.7B-InstructMath-v2', 'kyujinpy/Sakura-SOLRCA-Instruct-DPO', 'rishiraj/meow', 'fblgit/LUNA-SOLARkrautLM-Instruct', 'fblgit/UNAversal-8x7B-v1beta', 'NousResearch/Nous-Hermes-2-Yi-34B', 'mlabonne/NeuralMarcoro14-7B', 'Neuronovo/neuronovo-7B-v0.2', 'cloudyu/Mixtral_7Bx2_MoE', 'CultriX/MistralTrix-v1', 'cloudyu/Mixtral_7Bx5_MoE_30B', 'ryandt/MusingCaterpillar', 'SUSTech/SUS-Chat-34B', 'Sao10K/SOLAR-10.7B-NahIdWin', 'argilla/notus-8x7b-experiment', 'samir-fama/SamirGPT-v1', 'argilla/notux-8x7b-v1-epoch-2', 'argilla/notux-8x7b-v1', 'VAGOsolutions/SauerkrautLM-Mixtral-8x7B-Instruct', 'samir-fama/FernandoGPT-v1', 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'perlthoughts/Marcoroni-8x7B-v3-MoE', 'Brillibits/Instruct_Mixtral-8x7B-v0.1_Dolly15K', 'OpenBuddy/openbuddy-deepseek-67b-v15.2', 'mncai/yi-34B-v3', 'mncai/yi-34B-v2', 'CausalLM/72B-preview', 'rwitz/go-bruins-v2', 'DopeorNope/COKAL-v1-70B', 'bn22/Nous-Hermes-2-SOLAR-10.7B-MISALIGNED', 'rwitz/go-bruins', 'deepseek-ai/deepseek-llm-67b-chat', 'OpenBuddy/openbuddy-deepseek-67b-v15.1', 'migtissera/Tess-M-Creative-v1.0', 'bhenrym14/platypus-yi-34b', 'AA051610/A0106', 'dillfrescott/amadeus-v0.1', 'DiscoResearch/DiscoLM-70b', 'Walmart-the-bag/Solar-10.7B-Cato', 'aqweteddy/mistral_tv-neural-marconroni', 'Riiid/sheep-duck-llama-2-70b-v1.1', 'APMIC/caigun-lora-model-34B-v2', 'perlthoughts/neural-chat-v3-3-8x7b-MoE', 'kyujinpy/PlatYi-34B-Llama-Q', 'NExtNewChattingAI/shark_tank_ai_7_b', 'TomGrc/FusionNet_SOLAR', 'dillfrescott/Nous-Hermes-2-SOLAR-10.7B-x2-MoE', 'NousResearch/Nous-Hermes-2-SOLAR-10.7B', 'chargoddard/Yi-34B-Llama', '01-ai/Yi-34B-200K', 'one-man-army/una-neural-chat-v3-3-P2-OMA', 'ehartford/dolphin-2.2-70b', 'Q-bert/MetaMath-Cybertron', 'OpenPipe/mistral-ft-optimized-1227', 'KaeriJenti/kaori-70b-v1', 'Mihaiii/Pallas-0.2', 'HyperbeeAI/Tulpar-7b-v2', 'ValiantLabs/ShiningValiant', 'YeungNLP/firefly-mixtral-8x7b-v0.1', 'YeungNLP/firefly-mixtral-8x7b-v1', 'perlthoughts/Falkor-7b', 'AtAndDev/CapybaraMarcoroni-7B', 'APMIC/caigun-lora-model-34B-v3', 'v1olet/v1olet_merged_dpo_7B', 'migtissera/SynthIA-70B-v1.5', 'Mihaiii/Pallas-0.5-LASER-0.1', 'Mihaiii/Pallas-0.5', 'migtissera/Tess-34B-v1.4', 'Mihaiii/Pallas-0.4', 'Mihaiii/Pallas-0.3', 'ICBU-NPU/FashionGPT-70B-V1.1', 'Mihaiii/Pallas-0.5-LASER-0.2', 'Ba2han/BruinsV2-OpHermesNeu-11B', 'kyujinpy/PlatYi-34B-Q', 'Intel/neural-chat-7b-v3-3', 'migtissera/Tess-M-v1.1', 'migtissera/Tess-M-v1.3', 'jondurbin/bagel-34b-v0.2', 'fblgit/una-cybertron-7b-v2-bf16', 'openaccess-ai-collective/DPOpenHermes-7B-v2', 'fblgit/una-cybertron-7b-v1-fp16', 'DopeorNope/You_can_cry_Snowman-13B', 'MisterRid/saulgoodman-2x7b-alpha1', '01-ai/Yi-34B', 'MisterRid/saulgoodman-7b-alpha1', 'deepseek-ai/deepseek-llm-67b-base', 'OpenBuddy/openbuddy-deepseek-67b-v15-base', 'garage-bAInd/Platypus2-70B-instruct', 'PistachioAlt/Synatra-MCS-7B-v0.3-RP-Slerp', 'Mihaiii/Pallas-0.5-LASER-0.3', 'jondurbin/airoboros-l2-70b-2.2.1', 'itsliupeng/Mixtral-8x7B-v0.1-top3', 'Q-bert/Optimus-7B', 'chargoddard/loyal-piano-m7-cdpo', 'PulsarAI/Neural-una-cybertron-7b', 'psmathur/orca_mini_v3_70b', 'chargoddard/servile-harpsichord-cdpo', 'openchat/openchat-3.5-1210', 'Weyaxi/MetaMath-una-cybertron-v2-bf16-Ties', 'AA051610/FT', 'TokenBender/pic_7B_mistral_Full_v0.2', 'Weyaxi/OpenHermes-2.5-neural-chat-7b-v3-2-7B', 'adamo1139/Yi-34B-AEZAKMI-v1', 'chargoddard/loyal-piano-m7', 'mncai/agiin-13.6B-v0.0', 'jondurbin/spicyboros-70b-2.2', 'chargoddard/MixtralRPChat-ZLoss', 'psmathur/model_007', 'sequelbox/SpellBlade', 'psmathur/model_009', 'mistralai/Mixtral-8x7B-v0.1', 'psmathur/model_101', 'mncai/agiin-13.6B-v0.1', 'xDAN-AI/xDAN-L1-Chat-RL-v1', 'perlthoughts/Falkor-8x7B-MoE', 'Intel/neural-chat-7b-v3-2', 'Mihaiii/Pallas-0.5-LASER-0.4', 'Walmart-the-bag/WordWoven-13B', 'elinas/chronos007-70b', 'itsliupeng/llama2_70b_mmlu', 'decapoda-research/Antares-11b-v1', 'mlabonne/NeuralHermes-2.5-Mistral-7B', 'OrionStarAI/OrionStar-Yi-34B-Chat-Llama', 'Sao10K/Sensualize-Solar-10.7B', 'Azure99/blossom-v3_1-yi-34b', 'mncai/agiin-11.1B-v0.0', 'kyujinpy/PlatYi-34B-LoRA', 'Walmart-the-bag/openchat-3.5-Infinity', 'jondurbin/bagel-dpo-7b-v0.1', 'Mihaiii/Pallas-0.5-LASER-exp2-0.1', 'kyujinpy/PlatYi-34B-Llama-Q-v2', 'kyujinpy/PlatYi-34B-200k-Q-FastChat', 'tiiuae/falcon-180B', 'Weyaxi/OpenHermes-2.5-neural-chat-7b-v3-1-7B', 'VAGOsolutions/SauerkrautLM-Mixtral-8x7B', 'Q-bert/Bumblebee-7B', 'openaccess-ai-collective/DPOpenHermes-7B', 'Weyaxi/MetaMath-NeuralHermes-2.5-Mistral-7B-Linear', 'fangloveskari/ORCA_LLaMA_70B_QLoRA', 'Weyaxi/MetaMath-neural-chat-7b-v3-2-Ties', 'fblgit/juanako-7b-UNA', 'ehartford/Samantha-1.1-70b', 'beowolx/CodeNinja-1.0-OpenChat-7B', 'psmathur/test_42_70b', 'GOAT-AI/GOAT-70B-Storytelling', 'migtissera/Synthia-v3.0-11B', 'mlabonne/NeuralHermes-2.5-Mistral-7B-laser', 'diffnamehard/Mistral-CatMacaroni-slerp-uncensored', 'cognitivecomputations/dolphin-2.6-mistral-7b-dpo-laser', 'ehartford/Samantha-1.11-70b', 'Sao10K/Ana-v1-m7', 'Mihaiii/Pallas-0.5-LASER-0.5', 'cognitivecomputations/dolphin-2.6-mistral-7b-dpo', 'Weyaxi/neural-chat-7b-v3-1-OpenHermes-2.5-7B', 'berkeley-nest/Starling-LM-7B-alpha', 'psmathur/model_007_v2', 'perlthoughts/Starling-LM-alpha-8x7B-MoE', 'rishiraj/smol-7b', 'decem/Dionysus-Mistral-m3-v6', 'Weyaxi/MetaMath-NeuralHermes-2.5-Mistral-7B-Ties', 'meta-math/MetaMath-70B-V1.0', 'migtissera/Synthia-70B-v1.2b', 'Open-Orca/Mixtral-SlimOrca-8x7B', 'Walmart-the-bag/Misted-7B', 'TheBloke/neural-chat-7B-v3-2-GPTQ', 'migtissera/Synthia-70B-v1.2', 'migtissera/Synthia-70B-v1.1', 'Sao10K/Frostwind-10.7B-v1', 'migtissera/Synthia-70B', 'AA051610/A0105', 'Mihaiii/Pallas-0.5-LASER-0.6', 'uni-tianyan/Uni-TianYan', 'WizardLM/WizardMath-7B-V1.1', 'decapoda-research/Adrastea-7b-v1.0-dpo', 'NExtNewChattingAI/shark_tank_ai_7b_v2', 'openaccess-ai-collective/dpopenhermes-alpha-v0', 'rishiraj/uncensored', 'simonveitner/Math-OpenHermes-2.5-Mistral-7B', 'Brillibits/Instruct_Llama70B_Dolly15k', 'maywell/PiVoT-SOLAR-10.7B-RP', 'openaccess-ai-collective/openhermes-2_5-dpo-no-robots', 'augtoma/qCammel-70-x', 'augtoma/qCammel-70v1', 'augtoma/qCammel70', 'augtoma/qCammel-70', 'augtoma/qCammel-70x', 'garage-bAInd/Platypus2-70B', 'Xwin-LM/Xwin-LM-70B-V0.1', 'quantumaikr/llama-2-70b-fb16-orca-chat-10k', 'chargoddard/mistral-11b-slimorca', 'amazingvince/where-llambo-7b', 'seungduk/KoSOLAR-10.7B-v0.1', 'upstage/SOLAR-10.7B-v1.0', 'psmathur/model_51', 'TomGrc/FusionNet_passthrough', 'Undi95/Mixtral-4x7B-DPO-RPChat', 'meta-math/MetaMath-Mistral-7B', 'psmathur/model_420', 'TomGrc/FusionNet_passthrough_v0.1', 'tokyotech-llm/Swallow-70b-instruct-hf', 'mistralai/Mistral-7B-Instruct-v0.2', 'TheBloke/Mixtral-8x7B-v0.1-GPTQ', 'notbdq/alooowso', 'perlthoughts/Mistral-7B-Instruct-v0.2-2x7B-MoE', 'garage-bAInd/Camel-Platypus2-70B', 'NurtureAI/neural-chat-11b-v3-2', 'psmathur/model_42_70b', 'pankajmathur/Lima_Unchained_70b', 'jordiclive/Llama-2-70b-oasst-1-200', 'jondurbin/bagel-7b-v0.1', 'maywell/Synatra-10.7B-v0.4', 'wang7776/Mistral-7B-Instruct-v0.2-sparsity-10', 'Mihaiii/Metis-0.3', 'PistachioAlt/Noromaid-Bagel-7B-Slerp', 'OpenLemur/lemur-70b-chat-v1', '01-ai/Yi-34B-Chat', 'FPHam/Writing_Partner_Mistral_7B', 'llm-agents/tora-70b-v1.0', 'orangetin/OpenHermes-Mixtral-8x7B', 'simonveitner/MathHermes-2.5-Mistral-7B', 'cognitivecomputations/dolphin-2.2.1-mistral-7b', 'jondurbin/airoboros-l2-70b-gpt4-1.4.1', 'Weyaxi/OpenOrca-Zephyr-7B', 'NeuralNovel/Mistral-7B-Instruct-v0.2-Neural-Story', 'ehartford/dolphin-2.2.1-mistral-7b', 'cognitivecomputations/dolphin-2.6-mistral-7b', 'NousResearch/Nous-Puffin-70B', 'NeuralNovel/Panda-7B-v0.1', 'jarradh/llama2_70b_chat_uncensored', 'NeuralNovel/Tanuki-7B-v0.1', 'uukuguy/speechless-mistral-7b-dare-0.85', 'maywell/PiVoT-0.1-early', 'jondurbin/airoboros-l2-70b-gpt4-m2.0', 'OpenBuddy/openbuddy-mixtral-8x7b-v15.4', 'FinancialSupport/saiga-7b', 'perlthoughts/openchat-3.5-1210-32k', 'perlthoughts/openchat-3.5-1210-32k-8x7b-MoE', 'OpenAssistant/llama2-70b-oasst-sft-v10', 'hedronstone/OpenHermes-7B-Symbolic', 'hedronstone/OpenHermes-7B-Reasoner', 'elinas/chronos-70b-v2', 'NeuralNovel/Aeryth-7B-v0.1', 'maywell/PiVoT-10.7B-Mistral-v0.2', 'Neuronovo/neuronovo-7B-v0.1', 'jondurbin/airoboros-l2-70b-gpt4-2.0', 'Enoch/llama-65b-hf', 'KnutJaegersberg/CausalLM-Platypus-14B', 'jae24/openhermes_dpo_norobot_0201', 'TigerResearch/tigerbot-70b-base', 'Delcos/Starling-LM-11B-alpha', 'monology/openinstruct-mistral-7b', 'Azure99/blossom-v4-mistral-7b', 'luffycodes/higgs-llama-vicuna-ep25-70b', 'uukuguy/speechless-code-mistral-7b-v1.0', 'UCLA-AGI/test-test', 'perlthoughts/Falkor-16b', 'KaeriJenti/Kaori-34B-v1', 'SUSTech/SUS-Chat-72B', 'argilla/notus-7b-v1', 'maywell/Mini_Synatra_SFT', 'malhajar/Mistral-7B-v0.2-meditron-turkish', 'Deci/DeciLM-7B-instruct', 'liuda1/dm7b_sft_gpt88w_merge', 'KaeriJenti/kaori-34b-v3', 'decem/Dionysus-Mistral-m3-v5', 'Kiddyz/testllm-c2', 'TheBloke/Airoboros-L2-70B-2.1-GPTQ', 'adonlee/LLaMA_2_13B_SFT_v1', 'maywell/PiVoT-MoE', 'Azure99/blossom-v3-mistral-7b', 'teknium/CollectiveCognition-v1.1-Mistral-7B', 'UCLA-AGI/zephyr-7b-sft-full-spin-iter1', 'huggyllama/llama-65b', 'uukuguy/neural-chat-7b-v3-1-dare-0.85', 'Faradaylab/ARIA-70B-V3', 'migtissera/Tess-XS-v1-3-yarn-128K', 'Mihaiii/Metis-0.5', 'TheBloke/VicUnlocked-alpaca-65B-QLoRA-fp16', 'Azure99/blossom-v3_1-mistral-7b', 'ValiantLabs/ShiningValiantXS', 'jikaixuan/test_merged_model', 'Zardos/Kant-Test-0.1-Mistral-7B', 'willyninja30/ARIA-70B-French', 'UCLA-AGI/test0', 'jondurbin/airoboros-65b-gpt4-1.2', 'uukuguy/zephyr-7b-alpha-dare-0.85', 'migtissera/Tess-7B-v1.4', 'uukuguy/Orca-2-13b-f16', 'OpenLemur/lemur-70b-v1', 'uukuguy/speechless-mistral-dolphin-orca-platypus-samantha-7b-dare-0.85', 'kyujinpy/PlatYi-34B-200K-Q', 'migtissera/Synthia-7B-v3.0', 'athirdpath/Iambe-20b-DARE-v2', 'abhishek/zephyr-beta-math', 'microsoft/Orca-2-13b', 'HuggingFaceH4/zephyr-7b-beta', 'Faradaylab/ARIA-70B-V2', 'spmurrayzzz/Mistral-Syndicate-7B', 'proto-llm/uniwiz-7B-v0.1', 'Sao10K/Winterreise-m7', 'NeverSleep/Noromaid-7b-v0.2', 'jondurbin/bagel-8x7b-v0.2', 'Gryphe/MythoMist-7b', 'NickyNicky/Mistral-7B-OpenOrca-oasst_top1_2023-08-25-v2', 'Undi95/Llamix2-MLewd-4x13B', 'hiyouga/Qwen-14B-Chat-LLaMAfied', 'Intel/neural-chat-7b-v3-1']
len(models_names)

325

In [3]:
models_names = [models_names[i] for i in range(0,len(models_names),7)]
len(models_names)

47

In [4]:
models = []
for m in models_names:
    creator, model = tuple(m.split("/")) 
    models.append('open-llm-leaderboard/details_{:}__{:}'.format(creator, model))

In [8]:
models[:10]

['open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B',
 'open-llm-leaderboard/details_TomGrc__FusionNet',
 'open-llm-leaderboard/details_fblgit__UNA-SOLAR-10.7B-Instruct-v1.0',
 'open-llm-leaderboard/details_fblgit__LUNA-SOLARkrautLM-Instruct',
 'open-llm-leaderboard/details_cloudyu__Mixtral_7Bx5_MoE_30B',
 'open-llm-leaderboard/details_argilla__notux-8x7b-v1',
 'open-llm-leaderboard/details_mncai__yi-34B-v3',
 'open-llm-leaderboard/details_deepseek-ai__deepseek-llm-67b-chat',
 'open-llm-leaderboard/details_Walmart-the-bag__Solar-10.7B-Cato',
 'open-llm-leaderboard/details_TomGrc__FusionNet_SOLAR']

In [5]:
scenarios = ['harness_arc_challenge_25',
             'harness_hellaswag_10',
             #'harness_hendrycksTest_5',
             'harness_truthfulqa_mc_0',
             "harness_winogrande_5",
             "harness_gsm8k_5"]
      
mmlu_subscenarios = ['harness_hendrycksTest_abstract_algebra_5', 'harness_hendrycksTest_anatomy_5', 
                     'harness_hendrycksTest_astronomy_5', 'harness_hendrycksTest_business_ethics_5', 
                     'harness_hendrycksTest_clinical_knowledge_5', 'harness_hendrycksTest_college_biology_5', 
                     'harness_hendrycksTest_college_chemistry_5', 'harness_hendrycksTest_college_computer_science_5', 
                     'harness_hendrycksTest_college_mathematics_5', 'harness_hendrycksTest_college_medicine_5', 
                     'harness_hendrycksTest_college_physics_5', 'harness_hendrycksTest_computer_security_5', 
                     'harness_hendrycksTest_conceptual_physics_5', 'harness_hendrycksTest_econometrics_5', 
                     'harness_hendrycksTest_electrical_engineering_5', 'harness_hendrycksTest_elementary_mathematics_5', 
                     'harness_hendrycksTest_formal_logic_5', 'harness_hendrycksTest_global_facts_5', 
                     'harness_hendrycksTest_high_school_biology_5', 'harness_hendrycksTest_high_school_chemistry_5', 
                     'harness_hendrycksTest_high_school_computer_science_5', 'harness_hendrycksTest_high_school_european_history_5', 
                     'harness_hendrycksTest_high_school_geography_5', 'harness_hendrycksTest_high_school_government_and_politics_5', 
                     'harness_hendrycksTest_high_school_macroeconomics_5', 'harness_hendrycksTest_high_school_mathematics_5', 
                     'harness_hendrycksTest_high_school_microeconomics_5', 'harness_hendrycksTest_high_school_physics_5', 
                     'harness_hendrycksTest_high_school_psychology_5', 'harness_hendrycksTest_high_school_statistics_5',
                     'harness_hendrycksTest_high_school_us_history_5', 'harness_hendrycksTest_high_school_world_history_5', 
                     'harness_hendrycksTest_human_aging_5', 'harness_hendrycksTest_human_sexuality_5', 
                     'harness_hendrycksTest_international_law_5', 'harness_hendrycksTest_jurisprudence_5', 
                     'harness_hendrycksTest_logical_fallacies_5', 'harness_hendrycksTest_machine_learning_5', 
                     'harness_hendrycksTest_management_5', 'harness_hendrycksTest_marketing_5', 
                     'harness_hendrycksTest_medical_genetics_5', 'harness_hendrycksTest_miscellaneous_5',
                     'harness_hendrycksTest_moral_disputes_5', 'harness_hendrycksTest_moral_scenarios_5', 
                     'harness_hendrycksTest_nutrition_5', 'harness_hendrycksTest_philosophy_5', 
                     'harness_hendrycksTest_prehistory_5', 'harness_hendrycksTest_professional_accounting_5',
                     'harness_hendrycksTest_professional_law_5', 'harness_hendrycksTest_professional_medicine_5', 
                     'harness_hendrycksTest_professional_psychology_5', 'harness_hendrycksTest_public_relations_5',
                     'harness_hendrycksTest_security_studies_5', 'harness_hendrycksTest_sociology_5', 
                     'harness_hendrycksTest_us_foreign_policy_5', 'harness_hendrycksTest_virology_5', 
                     'harness_hendrycksTest_world_religions_5']


In [6]:
data = {}
for model in tqdm(models):
    data[model] = {}
    for s in mmlu_subscenarios+scenarios:
        data[model][s] = {}
        data[model][s]['correctness'] = None
        data[model][s]['dates'] = None

100%|██████████| 47/47 [00:00<00:00, 20074.57it/s]


In [7]:
skipped = 0
log = []
for model in tqdm(models):
    skipped_aux=0
    for s in mmlu_subscenarios+scenarios:
        if 'arc' in s: metric = 'acc_norm'
        elif 'hellaswag' in s: metric = 'acc_norm'
        elif 'truthfulqa' in s: metric = 'mc2'
        else: metric = 'acc'

        try:
            aux = load_dataset(model, s)
            data[model][s]['dates'] = list(aux.keys())
            data[model][s]['correctness'] = [a[metric] for a in aux['latest']['metrics']]
            print("\nOK {:} {:}\n".format(model,s))
            log.append("\nOK {:} {:}\n".format(model,s))
        except:
            try:
                aux = load_dataset(model, s)
                data[model][s]['dates'] = list(aux.keys())
                data[model][s]['correctness'] = aux['latest'][metric]
                print("\nOK {:} {:}\n".format(model,s))
                log.append("\nOK {:} {:}\n".format(model,s))
            except:
                data[model][s] = None
                print("\nSKIP {:} {:}\n".format(model,s))
                skipped_aux+=1
                log.append("\nSKIP {:} {:}\n".format(model,s))

    if skipped_aux>0: skipped+=1
        
    with open('leaderboard_raw_20240108.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    print("\nModels skipped so far: {:}\n".format(skipped))

  0%|          | 0/47 [00:00<?, ?it/s]


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_abstract_algebra_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_anatomy_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_astronomy_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_business_ethics_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_clinical_knowledge_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_college_biology_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_college_chemistry_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_college_computer_science_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_hendrycksTest_college_mathematics_5


OK open-llm-leaderboard/details_cloudyu__Mixtral_

  2%|▏         | 1/47 [02:23<1:49:51, 143.30s/it]


OK open-llm-leaderboard/details_cloudyu__Mixtral_34Bx2_MoE_60B harness_gsm8k_5


Models skipped so far: 0


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_abstract_algebra_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_anatomy_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_astronomy_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_business_ethics_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_clinical_knowledge_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_college_biology_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_college_chemistry_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_college_computer_science_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_hendrycksTest_college_mathematics_5


OK open-llm-leaderboard/details_TomGrc__FusionNet harness_

  4%|▍         | 2/47 [04:34<1:42:17, 136.40s/it]

...


OK open-llm-leaderboard/details_Undi95__Llamix2-MLewd-4x13B harness_winogrande_5



Generating 2023_12_24T22_02_51.116526 split: 0 examples [00:00, ? examples/s]

Generating latest split: 0 examples [00:00, ? examples/s]

100%|██████████| 47/47 [3:29:16<00:00, 267.16s/it]


OK open-llm-leaderboard/details_Undi95__Llamix2-MLewd-4x13B harness_gsm8k_5


Models skipped so far: 5

